Executive Summary
================
In this notebook I want to demonstrate a technique for detecting [lemmatisation](https://en.wikipedia.org/wiki/Lemmatisation) using a corpus of words from a target language.

Assumptions: 

* There are $C$ concepts (the lemmas).
* There are $T \ge C$ words in the vocabulary.
* A word belongs to 1 concept.
* A word is a transmission of a concept using a sequence of symbols (letters) which are garbled at error rate $\delta$ (spelling differences due to conjugations) and/or contain erasures (suffixes) at an erasure rate $\epsilon$.
* Group words into clusters using a weighted hamming metric to recover the lemmas.
* When words from the same concept are spelled differently, it's the vowels and not the consonants that change.

Problems to solve:

1. Determine $C$ from the vocabulary.
2. Determine the vowels in the alphabet.
2. Determine the garble $\delta$ and erasure $\epsilon$ rates from the vocabulary.

## Clustering and Error Correction

I want to try to do error correction in the following way: for each word $x_i$ I want to 

1. compute the probability that word $x_j$ is in the same cluster as word $x_i$: 
$$p_{ij} = Pr(x_j \in C_{x_i})$$
2. compute the probability that letter $k$ of word $x_i$ is $c$, 
$$Pr(x_{i,k} = c| \{x_j\})$$ given the probabilities $(p_{ij})$ and the corresponding words $(x_j)$.

To compute the first, I'll use Bayes' rule:
$$Pr_{post}(A) = Pr(A|B) =  \frac{Pr(A,B)}{Pr(B)} = \frac{ Pr(B|A)Pr_{prior}(A) }{Pr(B)}$$

When deciding that $x_j$ is in the same cluster as $x_i$ there are 2 opposing hypotheses to consider:

1. non-causal random hypothesis: word $x_j$ is not in the same cluster as word $x_i$; there is no correlation between the letters of $x_j$ and those of $x_i$.
2. causal hypothesis: word $x_j$ is in the same cluster as $x_i$; the letters of $x_j$ are correlated to those of $x_i$.

In the first hypothesis the prior probability is $(C-1)/C$ and for the second hypothesis it's $1/C$ since we're assuming the number of members of each cluster is approximately the same, and so cluster membership should be a uniform distribution. The odds for a second ball to fall into the same cluster bin as the first ball is only $1/C$ vs. $(C-1)/C$ falling into another bin.

So now I want to compute the ratio of the probabilities of the causal and random models given the observed $x_j$.

I have to be able to compute the probability of the similarity between $x_i$ and $x_j$.
Let $p_{l}$ denote the probability of letter $l$ occurring given the observed vocabulary.
Then the probability of corresponding letters matching in the random case are:

$$p_{random} := Pr(match) = \sum_l {p^2}_l$$

while in the causal case the probability of matching is

$$p_{causal} := (1-\delta)^2 + \delta^2 p_{random}$$

When comparing words $x_i$ and $x_j$ we define:

* $N := \min(|x_i|,|x_j| )$
* $A := \{t | x_{i,t} = x_{j,t}, \quad t \le N \}$
* $D := \{t | x_{i,t} \ne x_{j,t}, \quad t \le N  \}$
* $E := \{t | t>N\}$
* $q_{random} = 1- p_{random}$
* $q_{causal} = 1 - p_{causal}$

So to compute 
\begin{align}
L := \frac{Pr(causal|x_j)}{Pr(random|x_j)} &= \frac{ \frac{Pr(x_j|causal)Pr_{prior}(causal)}{Pr(x_j)}}{ \frac{Pr(x_j|random)Pr_{prior}(random)}{Pr(x_j)}} \\
&= \frac{ Pr(x_j|causal)Pr_{prior}(causal)}{ Pr(x_j|random)Pr_{prior}(random)} \\
&= \frac{ Pr(x_j|causal)}{Pr(x_j|random)}\frac{1/C}{(C-1)/C}\\
&= \frac{\binom{|A| + |D|}{|A|}p_{causal}^{|A|}q_{causal}^{|D|} }{\binom{|A| + |D|}{|A|}p_{random}^{|A|}q_{random}^{|D|} }\frac{1}{C-1}\\
&= \frac{p_{causal}^{|A|}q_{causal}^{|D|} }{p_{random}^{|A|}q_{random}^{|D|} }\frac{1}{C-1} = \frac{1}{C-1}\left(\frac{p_{causal}}{p_{random}}\right)^{|A|}\left(\frac{q_{causal}}{q_{random}}\right)^{|D|}
\end{align}

Now that I have a way to compute the ratio of probabilities $L$ of the 2 models given the observed word $x_j$ I can then compute the probability that $x_j$ is in the same cluster as $x_i$. Let $p_{i,j}$ denote the probability that $x_j$ is in the same cluster as $x_i$. Then we have:

\begin{align}
L &= \frac{p_{i,j}}{1-p_{i,j}} \\
(1-p_{i,j})L &= p_{i,j} \\
L &= (1+L)p_{i,j}\\
p_{i,j} &= \frac{L}{L+1}\\
\end{align}

## Recovering the lemmas
Once I have all the pairwise probabilities $\{p_{i,j}\}$ of belonging to the same cluster I have 2 strategies for recovering the clusters:

1. Construct the graph of words with an edge connecting words $x_i, x_j$ if $p_{i,j} > cutoff$ for some $0 < cutoff<1$ value. Then find the connected components. Each such component is a cluster. The cutoff value can be determined by modeling the values $\{\log(p_{i,j})\}$ as a mixture of 2 normal distributions, 
    * one for *pairs that are not in the same cluster* 
    * and the other for *pairs that are in the same cluster.*
    
2. Imposing an even stricter requirement on the graph, that all connected components need to be completely connected in order to be considered a cluster.

After obtaining the clusters, for each cluster $C_k$ we choose the shortest length word $x_i$ which maximizes 
$$score = \sum_{j \in C_k} \log\left(p_{i,j}\right)$$
as the lemma for the cluster.

## Recovering the erasure and garble rates from the data.
Recovering the MLE erasure rate is relatively straight forward. The erasures in a vector can be modelled by a binomial distribution $(n,\epsilon)$. One simply takes the sample mean of the observed erasures of the vectors to recover the erasure rate $\epsilon$.

To recover the garble rate $\delta$ it suffices to model the hamming weight distribution as a mixture of 2 models: the random and the causal.

Suppose we choose 2 words at random: $x_i,x_j$ and

* we determine which coordinates are mutually free of erasures. Call this number $K$.
* Count the number of coordinates in which they agree, $|A|$, and in which they disagree, $|D|$, with $|A| + |D| = K$.
* Compute the mean hamming bit distance:
$$hd_{i,j} = \frac{|D|}{|A| + |D|}$$

Can we determine the mean value $\mu_{hd}$ for all $hd_{i,j}$? Yes, yes we can.

Given 2 words $x_i,x_j$, after we've determined the erasure letters of $x_i, x_j$ we can model the remaining letters as being sampled from either 

* the random case with prior $(C-1)/C$ or 
* from the causal case with prior $1/C$. 

In the random case, the expected distribution for the hamming distance is independent of the garble rate while the expected distribution for the causal case will be a binomial distribution with parameters $(n = |A| +|D|,q)$ where $q_{causal} = 1 - p_{causal} = 1 - (1-\delta)^2 - \delta^2p_{random}$ which means 

* for the random case the mean hamming bit distance is $q_{random}$
* for the causal case the mean hamming bit distance is $q_{causal}$

So we can compute $\mu_{hd}$ like so:

\begin{align}
\frac{1}{T^2 -T}\sum_{x_i \ne x_j} hd_{i,j} &= \mu_{hd}\\
&= \mu_{hd,random}Pr(random) + \mu_{hd,causal}Pr(causal) \\
&= \mu_{hd,random}(C-1)/C + \mu_{hd,causal}(1/C) \\
&= q_{random}\times (C-1)/C + q_{causal}\times 1/C \\
\frac{1}{T^2 -T}\sum_{x_i \ne x_j} hd_{i,j} &= \mu_{hd} = \frac{q_{random}(C-1) + q_{causal}}{C}
\end{align}

So we can recover the parameter $q_{causal}$ by computing the sample mean value of the hamming letter distance $\mu_{hd}$:

$$ q_{causal} = C\mu_{hd} - (C-1)q_{random} $$
Once we have $q_{causal}$ we can solve for $\delta$.

### Recovering $C$
Recovering the estimate for $q_{causal}$ is predicated on knowledge of $C$. But we don't know $C$ yet. There are 2 possible techniques that I can think of for estimating $C$:

1. Estimate $C$ by recovering the suffixes in the language and group words by stem. The number of stem groups should be $\gtrapprox$ the number of lemma groups.

2. Maximize the separation of groups via $\{p_{ij}\}$ like so:
$$ score_{C} := \sum_{i \ne j} \max(p_{ij}, 1- p_{ij}) $$
For the correct $C$ we should have maximal separation of words into groups.


## Recovering the vowels in the alphabet
In languages that vowels are written and not assumed, we can try to recover the partition of letters into consonants and vowels by using an order 1 [hidden markov model](https://en.wikipedia.org/wiki/Hidden_Markov_model) (HMM) with 3 hidden states: whitespace, consonants, and vowels.

The idea:

1. There are $T$ words in the vocabulary.
2. Each word $w_i$ in the vocabulary is a sequence of letters: $(w_{i,t})$, some of the letters are consonants, some are vowels.
3. Concatenate using white space the vocabulary into  long sequence of words and spaces.
2. For the Hidden Markov Model there are 3 parameters to recover: 
    * $\pi_i :=Pr(X_0 = $consonant/vowel/white space$)$
    * $A_{rs} := Pr(X_{t+1} = s | X_{t} = r)$
    * $B_{r,s} := Pr(observed=s| X=r)$
    
Once we've recovered the optimal parameters $(\pi_i),A,B$ we can recover the assignment of consonants and vowels as follows. There are 3 hidden states: one of them represents consonants, the other represents vowels, the last white space.

$$B_{i,r} = Pr(\text{ letter }r | \text{hidden state }i)$$

If we apply the $\arg\max{}$ function to the rows of $B$, $\arg\max_{i} (B_{i,r})$ we get a map $$\arg\max_{i}(B_{i,r}):r \rightarrow \text{hidden state } 0,1,2$$

The white space hidden state has $B$ entry: $B(space) = 1$,  $B(letter) = 0$. The hidden state which has the most number of letters assigned to it are the consonants, and the other hidden state are the vowels.

### Modifying the Expectation-Maximization algorithm for HMM
In this [notebook](https://github.com/rpgomez/hidden_markov/blob/master/Hidden%20Markov%20Models.ipynb) I derive the steps for recovering a HMM from 1 observed sequence. We can modify the algorithm to handle multiple observed sequences or concatenate the vocabulary into 1 long stream interspersed with white space and 3 hidden states (consonant, vowel, white space) and
with the $B$ matrix initialized with entries:

1. $B_{consonant}(y) = \begin{cases} random & y \in alphabet \\ 0 & y = \text{whitespace}\end{cases}$
2. $B_{vowel}(y) = \begin{cases} random & y \in alphabet \\ 0 & y = \text{whitespace}\end{cases}$
3. $B_{whitespace} = \begin{cases} 0 & y \in alphabet \\ 1 & y = \text{whitespace}\end{cases}$

the $\pi$ array initialized with entries $[random, random,0]$

Then when we recover the MLE $\pi, A, B$ parameters we only focus on the first 2 rows of the $B$ matrix.

In [ ]:
%pylab inline

In [ ]:
from collections import Counter
from collections import defaultdict

from tqdm.notebook import tqdm

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import string

nltk.download('punkt')

try:
    from bs4 import BeautifulSoup
except ModuleNotFoundError:
    # We don't have BeautifulSoup installed. We also need the lxml module
    !pip install bs4 lxml
    from bs4 import BeautifulSoup


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pm_stemmer

In [ ]:
# download Mody Dick as html and then extract the content
!wget -c "https://www.gutenberg.org/files/2701/2701-h/2701-h.htm"
htmltxt= open('2701-h.htm','r').read()
soup = BeautifulSoup(htmltxt,'lxml')
content = soup.text.lower()

In [ ]:
content ="".join([x if x in string.ascii_letters else ' ' for x in content])
words = word_tokenize(content)
print("Length of word sequence: ",len(words))

vocabulary = set(words)
print("Size of vocabulary: ", len(vocabulary))

In [ ]:
vocabulary2 = list(vocabulary)
vocabulary2.sort()

We use 2 white spaces to join words to force unbias between consonants and vowels and white space in a first order HMM.

In [ ]:
one_string = "  ".join(vocabulary2) 

In [ ]:
alphabet =  list(set([x for x in one_string]))
alphabet.sort()
print(alphabet)

In [ ]:
import hmm

In [ ]:
B = np.random.dirichlet(np.ones(len(alphabet)),3)

B[2,:] = 0.0
B[2,0] = 1
B[:2,0] = 0.0
B[:2] = B[:2]/B[:2].sum(axis=1).reshape(-1,1)

my_pi = np.ones(3)*0.5
my_pi[2] = 0.0 # Not in the white space state

In [ ]:
my_hmm = hmm.hmm(num_hiddenstates=3,B =B.copy(),pi = my_pi.copy() )
my_hmm.fit(one_string[:100]) # to precompile hmm code.

In [ ]:
%%time
my_hmm = hmm.hmm(num_hiddenstates=3,B =B.copy(),pi = my_pi.copy() )
my_hmm.fit(one_string,verbose=True)

In [ ]:
title("Heat map of B matrix")
ylabel("Hidden State, 2 == white space")
xlabel("Observed letter, 0 == white space")
imshow(my_hmm.B,origin='lower',aspect='auto',interpolation='none'); colorbar();

In [ ]:
cluster_assignments = my_hmm.B.argmax(axis=0)

In [ ]:
accuracy = 1/3*my_hmm.B.max(axis=0).sum()
print("Probability of clustering letters into correct groups: {0:6.4f}%".format(100*accuracy))

In [ ]:
clusters = [[] for i in range(3)]
for y in range(my_hmm.B.shape[1]):
    letter = my_hmm.inv_map[y]
    assigned = cluster_assignments[y]
    clusters[assigned].append(letter)

for i in range(3):
    clusters[i].sort() 
    clusters[i] = ",".join(clusters[i])
    
print("Clusters:")
print("\n=====\n".join(clusters))

In [ ]:
if len(clusters[0]) < len(clusters[1]):
    vowels = clusters[0]
else:
    vowels = clusters[1]

In [ ]:
stemmer = pm_stemmer.Stemmer()
stemmer.fit(vocabulary2,verbose=True)

In [ ]:
import scipy.stats

In [ ]:
letter_prob = pm_stemmer.compute_alphabet_prob(vocabulary2)

In [ ]:
p_random = sum([p_l**2 for p_l in letter_prob.values()])
q_random = 1 - p_random
print("Probability of letters pairing up at random: ", p_random)

In [ ]:
def compute_hd(vocabulary):
    """Compute the mean hadamard symbol error rate."""
    
    # find the alphabet and translate it to ints so we can use numpy to speed things up.
    max_word_size = max([len(word) for word in vocabulary])

    alphabet = list(set([letter for letter in "".join(vocabulary)]))
    alphabet_to_ints = dict([(x,t) for t,x in enumerate(alphabet)])
    
    vocab = [(len(word),word) for word in vocabulary ]
    vocab.sort()
    word_vecs = np.zeros((len(vocab),max_word_size),dtype=int)

    for t,(l,word) in tqdm(enumerate(vocab),desc='generating word vectors'):
        for i in range(l):
            word_vecs[t,i] = alphabet_to_ints[word[i]]
    
    
    agreements = 0
    disagreements = 0
    for t in tqdm(range(len(vocab)-1),desc='cross comparing'):
        l,word = vocab[t]
        compare_me = word_vecs[t+1:,:l]-word_vecs[t,:l].reshape(1,-1)
        compare_me = (compare_me != 0).astype(int)
        local_disagreements = compare_me.sum()
        local_agreements = np.prod(compare_me.shape) - local_disagreements

        agreements += local_agreements
        disagreements += local_disagreements
#    mean_hd = hd_sum*2/(len(vocab)*(len(vocab)-1))
    mean_hd = disagreements/(agreements +disagreements)
    return mean_hd

In [ ]:
mean_hd = compute_hd(vocabulary2)

In [ ]:
mean_hd, q_random

In [ ]:
C = len(stemmer.stem_dict)
q_causal_probs = np.arange(C)*mean_hd - (np.arange(C)-1)*q_random

In [ ]:
q_causal, C_est  = q_causal_probs[q_causal_probs>0].min(),q_causal_probs[q_causal_probs>0].argmin()

In [ ]:
C_stem = len(set(stemmer.stem_dict.values()))

I'm not happy on how I've arrived at $q_{causal}$ and $C$. Let's see if I can use them now to group words.

In [ ]:
def sort_vocabulary(vocabulary):
    """Takes the list of words and sorts them by word length to make the
    computations easier and faster. """
    
    temp = [(len(word),word) for word in vocabulary]
    temp.sort()
    new_order = [x[1] for x in temp]
    return new_order

def compute_p_ij(vocabulary,q_random, q_causal,C):
    """Computes all possible pairings between vocabulary words.
    returns the array p_ij 
    
    CAVEAT: call this function with a sorted-by-word-length vocabulary
    
    I want to score as valid difference only those letter pairs
    that differ by vowels, i.e., (swim,swam) are fine to compare,
    but (abase,abate) are not.
    """
    
    p_causal = 1 - q_causal
    p_random = 1 - q_random
    
    p_ratio = (p_causal/p_random)
    q_ratio = (q_causal/q_random)
    one_c = 1/(C-1)
    
    T = len(vocabulary)
    max_word_size = len(vocabulary[-1])
    
    alphabet = list(set([letter for letter in "".join(vocabulary)]))
    alphabet_to_ints = dict([(x,t) for t,x in enumerate(alphabet)])

    vocab = [(len(word),word) for word in vocabulary]

    word_vecs = np.zeros((T,max_word_size),dtype=int)
    for t,(l,word) in tqdm(enumerate(vocab),desc='generating word vectors'):
        for i in range(l):
            word_vecs[t,i] = alphabet_to_ints[word[i]]

    p_ij = np.zeros((T,T),dtype=np.float32)
    
    for t in tqdm(range(len(vocab)-1),desc='cross comparing'):
        l,word = vocab[t]
        # I don't want to bother if the vector is not at least 4 characters long.
        if l < 5:
            continue
        compare_me = word_vecs[t+1:,:l]-word_vecs[t,:l].reshape(1,-1)
        compare_me = (compare_me != 0).astype(int)
        local_disagreements = compare_me.sum(axis=1)
        local_agreements = l - local_disagreements
        L = one_c*(p_ratio)**local_agreements*(q_ratio)**local_disagreements
        p_ij[t,t+1:] = L/(L+1)
        p_ij[t+1:,t] = L/(L+1)
        
    return p_ij

In [ ]:
def update_p_ij(p_ij, word1,word2,vowels):
    """Sets p_ij = 0 if the difference in word1 and word2 are (consonant, consonant)
    or (consonant, vowel) pairs."""
    
    lengths = [(len(word1),word1),(len(word2),word2)]
    lengths.sort()
    wor1,wor2 = lengths[0][1], lengths[1][1]
    
    for t in range(len(wor1)):
        if wor1[t] != wor2[t]:
            if not wor1[t] in vowels:
                return 0.0
            if not wor2[t] in vowels:
                return 0.0
            
    return p_ij

In [ ]:
vocabulary2 = sort_vocabulary(vocabulary2)

In [ ]:
p_ij = compute_p_ij(vocabulary2,q_random, q_causal,C_stem)

In [ ]:
possible_pairing = np.argwhere(p_ij > 0.8)
print("# of pairings to consider:", len(possible_pairing))

In [ ]:
for (i,j) in tqdm(possible_pairing,desc='second filter'):
    word1, word2 = vocabulary2[i], vocabulary2[j]
    update_score = update_p_ij(p_ij[i,j],word1,word2,vowels)
    p_ij[i,j] = update_score

In [ ]:
possible_pairing = np.argwhere(p_ij > 0.8)
print("# of pairings to consider:", len(possible_pairing))

In [ ]:
temp = p_ij.flatten()

In [ ]:
hist(temp[temp>0.1],bins=linspace(0.1,1,100));

In [ ]:
(temp>0.8).sum()/temp.shape[0] * len(vocabulary2), (temp>0.8).sum()

In [ ]:
possible = np.argwhere(p_ij>0.8)

In [ ]:
import networkx as nx

In [ ]:
node_names = [vocabulary2[a] for a,b in possible] + [vocabulary2[b] for a,b in possible]
nodes = set(node_names)
print(len(nodes), len(vocabulary2))

In [ ]:
G = nx.Graph([(vocabulary2[a],vocabulary2[b]) for (a,b) in possible])

In [ ]:
clusters = [x for x in nx.connected_components(G)]

In [ ]:
len(clusters)

In [ ]:
cliques =[x for x in nx.find_cliques(G)]

In [ ]:
bincount([len(x) for x in cliques])

In [ ]:
big_cliques = [x for x in cliques if len(x) == 5]

In [ ]:
for x in big_cliques[0]:
    print(x, stemmer.stem(x))

In [ ]:
for x in cliques[:10]:
    print(x)

In [ ]:
stemmer.stem('constantly'), stemmer.stem('extravaganzas')

In [ ]:
cliques[1]

In [ ]:
for t in range(100):
    print("{0:02}".format(t), " \t".join(list(clusters[t])[:5]))

In [ ]:
clusters[1]

In [ ]:
for a,b in possible[:30]:
    print(vocabulary2[a],vocabulary2[b])

In [ ]:
for a,b in possible[-30:]:
    print(vocabulary2[a],vocabulary2[b])